In [ ]:
# Add this cell after cell7
 
session.sql("USE ROLE CO2_ROLE").collect()

session.sql("USE WAREHOUSE CO2_WH").collect()

session.sql("USE DATABASE CO2_DB").collect()
 
# Make sure the session uses the RAW_CO2 schema to create the procedure

session.use_schema('CO2_DB.RAW_CO2')
 
# Create the CO2 data processing stored procedure

session.sql("""

CREATE OR REPLACE PROCEDURE load_co2_data()

RETURNS STRING

LANGUAGE PYTHON

RUNTIME_VERSION = '3.8'

PACKAGES = ('snowflake-snowpark-python')

HANDLER = 'main'

AS

$$

def main(session):

    try:

        # Set up environment - The USE commands are no longer necessary here

        #session.sql("USE ROLE ACCOUNTADMIN").collect()  # Remove this line

        #session.sql("USE DATABASE CO2_DB").collect()   # Remove this line

        #session.sql("USE SCHEMA RAW_CO2").collect()     # Remove this line
 
        # Create the table

        session.sql('''

            CREATE OR REPLACE TABLE RAW_CO2.Daily_Measurements (

                date STRING,

                co2_ppm FLOAT

            );

        ''').collect()
 
        # Grant privileges

        session.sql('''

            GRANT ALL PRIVILEGES ON TABLE RAW_CO2.Daily_Measurements TO ROLE ACCOUNTADMIN;

        ''').collect()
 
        # Copy data from stage

        session.sql('''

            COPY INTO RAW_CO2.Daily_Measurements

            FROM @RAW_CO2.CO2_EXTERNAL_STAGE

            FILE_FORMAT = (

                TYPE = 'CSV'

                SKIP_HEADER = 1

                FIELD_OPTIONALLY_ENCLOSED_BY = '"'

            )

            ON_ERROR = 'CONTINUE';

        ''').collect()
 
        # Create stream

        session.sql('''

            CREATE OR REPLACE STREAM RAW_CO2.DAILY_MEASUREMENTS_STREAM

            ON TABLE RAW_CO2.Daily_Measurements;

        ''').collect()
 
        return "Raw data loading completed successfully"

    except Exception as e:

        return f"Error: {str(e)}"

$$

""").collect()

 